In [1]:
import numpy as np
import igl
from scipy.sparse.linalg import spsolve
import meshplot as mp
from meshplot import plot, subplot, interact

In [2]:
back = '/Users/xuzhaoyang/Desktop/back.off'
back1 = '/Users/xuzhaoyang/Desktop/back1.off'
back2 = '/Users/xuzhaoyang/Desktop/back2.off'

'''
def read_off(path):
    

    Parameters
    ----------
    path : path
        file.off

    Returns
    -------
    U : array
        rgb array of whole patch
    U_tilde : array
        random change 5% rgb of whole patch

    

    with open(path, 'r') as f:
        a = f.read().split()
        b = a[4:] # remove 'COFF' and other lines
        
        # list to array
        file_float = []
        for num in b:
            file_float.append(float(num))
        c = np.array(file_float)
        d = c.reshape(-1,7)
        
        # only keep rgb lines, remove the index of vertex lines
        ind = list(d[:,0]).index(3)
        vertex = d[0:ind,0:3]
        rgba = d[0:ind,3:7] # include alpha, RGBA
        U = d[0:ind,3:6]
        
        p,d = U.shape # p = n, d = 3
        
        # randomly choose 5% from the whole patch, and change them
        ran_index = np.random.random_integers(0,p-1,size = int(p*0.05))
        U_tilde = U
        for i in ran_index:
            U_tilde[i] = np.random.random_integers(1,255,size=3)
            
    return vertex,U,U_tilde
'''

def read_off(path):

    with open(path, 'r') as f:
        a = f.read().split()
        b = a[4:] # remove 'COFF' and other lines
        
        # list to array
        file_float = []
        for num in b:
            file_float.append(float(num))
        c = np.array(file_float)
        d = c.reshape(-1,7)
        
        # only keep rgb lines, remove the index of vertex lines
        ind = list(d[:,0]).index(3)
        vertex = d[0:ind,0:3]
        U = d[0:ind,3:6]
        
            
    return vertex,U

V1,U1 = read_off(back1)
V2,U2 = read_off(back2)
V,U = read_off(back)

p1,_ = V1.shape
p2,_ = V2.shape
p,_ = V.shape

v1_ind = []

for i in range(0,p1):
    for j in range(0,p):
        if np.all(V1[i] == V[j]):
            v1_ind.append(j)
            #print(j)
            break
#print(v1_ind)

v2_ind = []
test = []
for i in range(0,p2):
    for j in range(0,p):
        if np.all(V2[i] == V[j]):
            #print(i)
            test.append(i)
            v2_ind.append(j)
            #print(j)
            break

#print(v2_ind)

#V2[786]

'''
inter = list(set(v1_ind)&set(v2_ind))
            
U1_delta = U1-U1_tilde
U2_delta = U2-U2_tilde
    

term1 = np.zeros((p,3))
for i in range (0,len(v1_ind)):
    term1[v1_ind[i]] = U1_delta[i]

term2 = np.zeros((p,3))
for i in range (0,len(v2_ind)):
    term2[v2_ind[i]] = U2_delta[i]


term = term1**2 + term2**2

for i in inter:
    term[i] = term[i]/2

R = term[:,0]
G = term[:,1]
B = term[:,2]

'''

'\ninter = list(set(v1_ind)&set(v2_ind))\n            \nU1_delta = U1-U1_tilde\nU2_delta = U2-U2_tilde\n    \n\nterm1 = np.zeros((p,3))\nfor i in range (0,len(v1_ind)):\n    term1[v1_ind[i]] = U1_delta[i]\n\nterm2 = np.zeros((p,3))\nfor i in range (0,len(v2_ind)):\n    term2[v2_ind[i]] = U2_delta[i]\n\n\nterm = term1**2 + term2**2\n\nfor i in inter:\n    term[i] = term[i]/2\n\nR = term[:,0]\nG = term[:,1]\nB = term[:,2]\n\n'

In [3]:
v,f = igl.read_triangle_mesh(back)
L0 = igl.cotmatrix(v, f)
lamda = 0.01
L1 = 0.01*L0
L2 = 1*L0
L3 = 10*L0
L4 = 100*L0

In [4]:
#insect = '/Users/xuzhaoyang/Desktop/graphosoma/Graphosoma.obj'

## Construct Laplacian and mass matrix
'''
R1 = U1_tilde[:,0]
G1 = U1_tilde[:,1]
B1 = U1_tilde[:,2]

R2 = U2_tilde[:,0]
G2 = U2_tilde[:,1]
B2 = U2_tilde[:,2]
'''
R1 = U1[:,0]*0.5
G1 = U1[:,1]*0.5
B1 = U1[:,2]*0.5

R2 = U2[:,0]
G2 = U2[:,1]
B2 = U2[:,2]

X_1 = np.zeros((p,p))
for i in v1_ind:
    X_1[i][i]=1
    
X_2 = np.zeros((p,p))
for i in v2_ind:
    X_2[i][i]=1
    
X_all = X_1+X_2

def b(RGB1,RGB2,v1_ind,v2_ind,p1,p2):
    
    X_u1 = np.zeros((p,p))
    for i in range(0,p1):
        ind = v1_ind[i]
        X_u1[ind][ind] = RGB1[i]
        
    X_u2 = np.zeros((p,p))
    for i in range(0,p2):
        indd = v2_ind[i]
        X_u2[indd][indd] = RGB2[i]
    
    Xu = X_u1+X_u2
    b_uv = np.zeros(p)
    for i in range(0,p):
        b_uv[i] = Xu[i][i]
    
    return b_uv

R_all = b(R1,R2,v1_ind,v2_ind,p1,p2)
G_all = b(G1,G2,v1_ind,v2_ind,p1,p2)
B_all = b(B1,B2,v1_ind,v2_ind,p1,p2)
    
RGB = np.vstack([R_all,G_all,B_all]).T
A1 = X_all - L1
A2 = X_all - L2
A3 = X_all - L3
A4 = X_all - L4
x1 = np.linalg.solve(A1,RGB)
x2 = np.linalg.solve(A2,RGB)
x3 = np.linalg.solve(A3,RGB)
x4 = np.linalg.solve(A4,RGB)

In [5]:
#print(x)

In [6]:
#print(U)

In [7]:
'''
def read_off(path):

    with open(path, 'r') as f:
        a = f.read().split()
        b = a[4:] # remove 'COFF' and other lines
        
        # list to array
        file_float = []
        for num in b:
            file_float.append(float(num))
        c = np.array(file_float)
        d = c.reshape(-1,7)
        
        # only keep rgb lines, remove the index of vertex lines
        ind = list(d[:,0]).index(3)
        vertex = d[0:ind,0:3]
        U = d[0:ind,3:6]
        
            
    return vertex,U
    '''

"\ndef read_off(path):\n\n    with open(path, 'r') as f:\n        a = f.read().split()\n        b = a[4:] # remove 'COFF' and other lines\n        \n        # list to array\n        file_float = []\n        for num in b:\n            file_float.append(float(num))\n        c = np.array(file_float)\n        d = c.reshape(-1,7)\n        \n        # only keep rgb lines, remove the index of vertex lines\n        ind = list(d[:,0]).index(3)\n        vertex = d[0:ind,0:3]\n        U = d[0:ind,3:6]\n        \n            \n    return vertex,U\n    "

In [8]:
_,U=read_off(back)

In [9]:
d = mp.subplot(v, f, c=v[:, 1], s=[3, 2, 0])
mp.subplot(v, f, c=(U/255), s=[3, 2, 1], data=d)
mp.subplot(v, f, c=(x1/255), s=[3, 2, 2], data=d)
mp.subplot(v, f, c=(x2/255), s=[3, 2, 3], data=d)
mp.subplot(v, f, c=(x3/255), s=[3, 2, 4], data=d)
mp.subplot(v, f, c=(x4/255), s=[3, 2, 5], data=d)